In [5]:
import json
import requests
import re

In [3]:
url = "https://api.iextrading.com/1.0/ref-data/symbols"

headers = {'cache-control': 'no-cache'}

response = requests.request("GET", url, headers=headers)

data = json.loads(response.text)

In [62]:
name_symbol_map = {}
symbo_name_map = {}

for company in data:
    symbol = company['symbol']
    name = company['name']
    name = re.sub(r'[^a-zA-Z ]',r'', name).lower()
    name = re.sub(' +', ' ',name)
    if len(name) != 0:
        name_symbol_map[name] = symbol
    
    symbo_name_map[symbol] = name

In [18]:
name_symbol_map

{'agilent technologies inc': 'A',
 'alcoa corporation': 'AA',
 'perth mint physical gold': 'AAAU',
 'altaba inc': 'AABA',
 'aac holdings inc': 'AAC',
 'advisorshares dorsey wright adr': 'AADR',
 'american airlines group inc': 'AAL',
 'altisource asset management corp com': 'AAMC',
 'atlantic american corporation': 'AAME',
 'aarons inc': 'AAN',
 'applied optoelectronics inc': 'AAOI',
 'aaon inc': 'AAON',
 'advance auto parts inc wi': 'AAP',
 'apple inc': 'AAPL',
 'american assets trust inc': 'AAT',
 'almaden minerals ltd': 'AAU',
 'atlas air worldwide holdings': 'AAWW',
 'ishares msci all country asia ex japan index fund': 'AAXJ',
 'axon enterprise inc': 'AAXN',
 'alliancebernstein holding lp units': 'AB',
 'renmin tianli group inc': 'ABAC',
 'abb ltd': 'ABB',
 'abbvie inc': 'ABBV',
 'amerisourcebergen corporation': 'ABC',
 'ameris bancorp': 'ABCB',
 'alcentra capital corp': 'ABDC',
 'abeona therapeutics inc': 'ABEO',
 'ambev sa american depositary shares each representing ': 'ABEV',
 '

In [53]:
single_words_dict = {}

# for each company name
for key in name_symbol_map:
    # for each word in the comapny name
    for word in key.split(" "):
        if word in single_words_dict:
            arr = single_words_dict[word]
        else:
            # first time appending the word
            arr = []
        arr.append({ 'index' : key, 'weight' : (len(word) / len(key)) })
        single_words_dict[word] = arr
        
single_words_dict['apple']

[{'index': 'apple inc', 'weight': 0.5555555555555556},
 {'index': 'apple hospitality reit inc', 'weight': 0.19230769230769232}]

In [57]:
def get_symbol(tweet):
    # strip tweet from non english chars
    tweet = re.sub(r'[^a-zA-Z ]',r'', tweet).lower()
    tweet = re.sub(' +', ' ',tweet)
    
    score_map = {}
    
    # for each word
    for word in tweet.split(" "):
        if word in single_words_dict:
            # look up results
            lookup_res = single_words_dict[word]
            
            for company_index in lookup_res:
                company_name = company_index['index']
                company_index_score = company_index['weight']
                
                if company_name in score_map:
                    score_map[company_name] += company_index_score
                else:
                    score_map[company_name] = company_index_score
    
    maxi = 0
    maxc = None
    total = 0
    for key,val in score_map.items():
        total += val
        
        if val > maxi:
            maxc = key
            maxi = val
            
    if maxc is not None:
        return (name_symbol_map[maxc], maxi)
    
tweet = "GitHub finally has unlimited free repositories! Well done Microsoft corporation!"


get_symbol(tweet)


('MSFT', 0.9523809523809523)

In [59]:
get_symbol('apple')

('AAPL', 0.5555555555555556)

In [65]:
get_symbol('alphabet')

('GOOGL', 0.6666666666666666)

In [66]:
symbo_name_map['GOOGL']

'alphabet inc'